In [ ]:
!pip install transformers
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 56.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3


In [ ]:
!pip install --upgrade praatio
!pip install praatio

In [ ]:
!tar -xzvf "/content/test-clean.tar.gz"

LibriSpeech/LICENSE.TXT
LibriSpeech/README.TXT
LibriSpeech/CHAPTERS.TXT
LibriSpeech/SPEAKERS.TXT
LibriSpeech/BOOKS.TXT
LibriSpeech/test-clean/
LibriSpeech/test-clean/6930/
LibriSpeech/test-clean/6930/75918/
LibriSpeech/test-clean/6930/75918/6930-75918-0000.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0007.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0006.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0005.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0004.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0002.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0011.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0015.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0003.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0010.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0001.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0018.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0008.flac
LibriSpeech/test-clean/6930/75918/6930-75918-0017.flac
LibriSpeech/test-clean/

In [ ]:
import os
import zipfile  # zipfile 모듈을 가져옵니다.

# 압축 파일 경로 (업로드한 파일의 경로에 따라 변경)
zip_file_path = '/content/archive.zip'
output_dir = '/content/dataset'

# 압축 해제
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print("압축 해제가 완료되었습니다.")

# 데이터셋 구조 확인
for root, dirs, files in os.walk(output_dir):
    print(root, len(files), "개의 파일")

압축 해제가 완료되었습니다.
/content/dataset 1 개의 파일
/content/dataset/Female_dysarthria 1 개의 파일
/content/dataset/Female_dysarthria/F03 1 개의 파일
/content/dataset/Female_dysarthria/F03/Session3 0 개의 파일
/content/dataset/Female_dysarthria/F03/Session3/Txt 158 개의 파일
/content/dataset/Female_dysarthria/F03/Session3/Wav 158 개의 파일
/content/dataset/Female_dysarthria/F03/Session2 0 개의 파일
/content/dataset/Female_dysarthria/F03/Session2/Txt 129 개의 파일
/content/dataset/Female_dysarthria/F03/Session2/Wav 129 개의 파일
/content/dataset/Female_dysarthria/F03/Session1 1 개의 파일
/content/dataset/Female_dysarthria/F03/Session1/Txt 164 개의 파일
/content/dataset/Female_dysarthria/F03/Session1/Wav 164 개의 파일
/content/dataset/Female_dysarthria/F01 0 개의 파일
/content/dataset/Female_dysarthria/F01/Session1 1 개의 파일
/content/dataset/Female_dysarthria/F01/Session1/Txt 93 개의 파일
/content/dataset/Female_dysarthria/F01/Session1/Wav 93 개의 파일
/content/dataset/Female_dysarthria/F04 0 개의 파일
/content/dataset/Female_dysarthria/F04/Session2 0 개의 파일
/

In [ ]:
import os
import torch
from sklearn.model_selection import train_test_split
import numpy as np
import librosa

# 데이터 로드 함수
def load_data_from_path(directory):
    metadata = []
    for root, _, files in os.walk(directory):
        # 경로에서 성별과 Dysarthria 여부 추출
        parts = root.split("/")  # "/" 기준으로 분리
        if len(parts) <= 5:  # 유효하지 않은 경로는 무시
            continue
        folder_name = parts[3]  # 기준 폴더명 추출 (6번째 요소)
        if "Female" in folder_name:
            gender = "Female"
        elif "Male" in folder_name:
            gender = "Male"
        else:
            continue

        if "Non_Dysarthria" in folder_name:
            dysarthria = "Non_Dysarthria"
        else:
            dysarthria = "Dysarthria"
        # "Wav" 폴더에서만 오디오 파일 로드
        if "Wav" in root:
            for file in files:
                if file.endswith(".wav"):
                    audio_path = os.path.join(root, file)
                    text_path = audio_path.replace("Wav", "Txt").replace(".wav", ".txt")
                    if os.path.exists(text_path):
                        with open(text_path, "r", encoding="utf-8") as f:
                            text = f.read().strip()
                        audio_array, sampling_rate = librosa.load(audio_path, sr=None)  # 원래 샘플링 레이트 유지
                        metadata.append({
                            "audio": {
                                "path": audio_path,
                                "array": audio_array,
                                "sampling_rate": sampling_rate,
                            },
                            "sentence": text,
                            "gender": gender,
                            "dysarthria": dysarthria
                        })
    return metadata
# 데이터 로드
data_directory = "/content/dataset"
metadata = load_data_from_path(data_directory)

# Dysarthria와 Non_Dysarthria 데이터 분리
non_dysarthria_data = [item for item in metadata if item["dysarthria"] == "Non_Dysarthria"]
dysarthria_data = [item for item in metadata if item["dysarthria"] == "Dysarthria"]

In [ ]:
import os
import librosa
import numpy as np
import soundfile as sf
from praatio import textgrid
from tqdm import tqdm

# 음소 구간 추출 함수
def extract_phoneme_segments(audio_array, sampling_rate):
    # 짧은 구간의 에너지를 계산
    frame_length = int(sampling_rate * 0.025)  # 25ms
    hop_length = int(sampling_rate * 0.01)  # 10ms
    energy = librosa.feature.rms(y=audio_array, frame_length=frame_length, hop_length=hop_length)[0]

    # 에너지 변화에서 피크 감지
    peaks, _ = find_peaks(energy, height=np.percentile(energy, 75), distance=5)
    segments = librosa.frames_to_time(peaks, sr=sampling_rate, hop_length=hop_length)
    return segments

# TextGrid 생성 함수
def create_textgrid(transcript, output_path, duration):
    words = transcript.split()
    num_words = len(words)
    interval_duration = duration / num_words

    entries = []
    start_time = 0.0

    for word in words:
        end_time = start_time + interval_duration
        entries.append((start_time, end_time, word))
        start_time = end_time

    tg = textgrid.Textgrid()
    tg.addTier(textgrid.IntervalTier("phonemes", entries, 0, duration))
    tg.save(output_path, format="long_textgrid", includeBlankSpaces=True)
    return output_path

# TextGrid에서 음소 구간 추출
def extract_phoneme_intervals(textgrid_path):
    tg = textgrid.openTextgrid(textgrid_path, includeEmptyIntervals=False)
    phoneme_tier = tg.getTier("phonemes")
    phoneme_intervals = phoneme_tier.entries
    return phoneme_intervals

# Dysarthria 변형 적용
def apply_dysarthria_transform(audio_path, phoneme_intervals, sr=16000):
    audio, _ = librosa.load(audio_path, sr=sr)
    augmented_audio = []

    for interval in phoneme_intervals:
        start_sample = int(interval[0] * sr)
        end_sample = int(interval[1] * sr)
        segment = audio[start_sample:end_sample]

        if len(segment) == 0:
            continue

        # Dysarthria 효과: 시간 축소
        stretched_segment = librosa.effects.time_stretch(segment, rate=0.5)
        augmented_audio.extend(stretched_segment)

    augmented_audio = np.array(augmented_audio, dtype=np.float32)
    return augmented_audio

# Non-Dysarthria 데이터 증강 함수
def augment_non_dysarthria(non_dysarthria_data):
    augmented_data = []
    for item in tqdm(non_dysarthria_data):
        audio = item["audio"]
        transcript = item["sentence"]

        # TextGrid 생성
        audio_duration = librosa.get_duration(y=audio["array"], sr=audio["sampling_rate"])
        textgrid_path = os.path.join("/tmp", "temp.TextGrid")
        create_textgrid(transcript, textgrid_path, audio_duration)

        # TextGrid에서 음소 구간 추출
        phoneme_intervals = extract_phoneme_intervals(textgrid_path)

        # Dysarthria 변형 적용
        augmented_audio = apply_dysarthria_transform(audio["path"], phoneme_intervals)

        # 결과 저장
        augmented_data.append({
            "audio": {
                "path": audio["path"],
                "array": augmented_audio,
                "sampling_rate": audio["sampling_rate"]
            },
            "sentence": transcript,
            "gender": item["gender"],
            "dysarthria": "Augmented"
        })
    return augmented_data

# Dysarthria 데이터에서 음소 구간 추출
print("Dysarthria 데이터에서 음소 구간 추출 중...")
dysarthria_segments = []
for item in tqdm(dysarthria_data):
    audio = item["audio"]
    audio_array = audio["array"]
    sampling_rate = audio["sampling_rate"]
    segments = extract_phoneme_segments(audio_array, sampling_rate)
    dysarthria_segments.extend(segments)

# Non-Dysarthria 데이터 증강
print("Non-Dysarthria 데이터 증강 중...")
augmented_data = augment_non_dysarthria(non_dysarthria_data)

# 결과 확인
print(f"증강된 데이터 개수: {len(augmented_data)}")

# 증강 데이터 저장
output_dir = "/content/augmented_dataset"
os.makedirs(output_dir, exist_ok=True)
for i, item in enumerate(augmented_data):
    audio_path = os.path.join(output_dir, f"augmented_{i}.wav")
    sf.write(audio_path, item["audio"]["array"], item["audio"]["sampling_rate"])
    with open(audio_path.replace(".wav", ".txt"), "w", encoding="utf-8") as f:
        f.write(item["sentence"])

print("증강 데이터 저장이 완료되었습니다.")


Dysarthria 데이터에서 음소 구간 추출 중...


100%|██████████| 1028/1028 [00:03<00:00, 298.95it/s]


Non-Dysarthria 데이터 증강 중...


100%|██████████| 1868/1868 [01:52<00:00, 16.53it/s]


증강된 데이터 개수: 1868
증강 데이터 저장이 완료되었습니다.


In [ ]:
from google.colab import files
import shutil
import os

# 압축할 폴더 경로와 출력 zip 파일 경로
folder_to_zip = "/content/augmented_dataset"  # 다운로드할 폴더
output_zip = "/content/augment_data.zip"  # zip 파일 이름과 경로

# 폴더를 zip으로 압축
shutil.make_archive(base_name=output_zip.replace(".zip", ""), format="zip", root_dir=folder_to_zip)

print(f"Folder {folder_to_zip} has been compressed to {output_zip}")
# 압축된 zip 파일 다운로드
files.download(output_zip)

Folder /content/augmented_dataset has been compressed to /content/augment_data.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>